In [1]:
from gensim.models.word2vec import Word2Vec
import numpy as np
import pandas as pd

unable to import 'smart_open.gcs', disabling that module


In [3]:
model = Word2Vec.load('./models/300feature_490minwords_10context')

In [5]:
model['flower'].shape

/home/rohit/.pyenv/versions/3.6.10/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


(300,)

In [20]:
def make_feature_vector(words, model, no_of_features):
    feature_vec = np.zeros((no_of_features,), dtype="float32")
    n_words = 0
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
            
    feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

In [10]:
def get_feature_vectors(reviews, model, no_of_features):
    count = 0
    feature_vec = np.zeros((len(reviews), no_of_features), dtype='float32')
    for review in reviews:
        if count%1000 == 0:
            print ("Review %d of %d" % (count, len(reviews)))
            
        feature_vec[count] = make_feature_vector(reviews[count], model, no_of_features)
        count += 1
    return feature_vec

### Preparing data

In [17]:
import re
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

In [11]:
train = pd.read_csv('data/labeledTrainData.tsv', header=0, delimiter='\t', quoting=3)

In [12]:
test = pd.read_csv('data/testData.tsv', header=0, delimiter='\t', quoting=3)

In [13]:
def review_to_wordlist(review, remove_stopwords=False):
    review_text = BeautifulSoup(review).get_text()
    review_text = re.sub("[^a-zA-Z]", ' ', review_text)
    words = review_text.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words('english'))
        words = [w for w in words if not w in stops]
    return words

### Training

In [18]:
clean_train_reviews = []
for review in train['review']:
    clean_train_reviews.append(review_to_wordlist(review, remove_stopwords = True))

In [21]:
train_data_vectors = get_feature_vectors(clean_train_reviews, model, 300)

Review 0 of 25000


/home/rohit/.pyenv/versions/3.6.10/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000


In [22]:
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist(review, remove_stopwords=True))

test_data_vectors = get_feature_vectors(clean_test_reviews, model, 300)

Review 0 of 25000


/home/rohit/.pyenv/versions/3.6.10/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000


In [23]:
import pickle

In [25]:
pickle.dump(train_data_vectors, open('./data/clean_train_embedded.pkl', 'wb'))

In [26]:
pickle.dump(test_data_vectors, open('./data/clean_test_embedded.pkl', 'wb'))

### Model

In [27]:
from sklearn.ensemble import RandomForestClassifier

In [28]:
rf = RandomForestClassifier()

In [30]:
rf.fit(train_data_vectors, train['sentiment'])

/home/rohit/.pyenv/versions/3.6.10/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [33]:
# Shit, again didnt split into train test

In [34]:
# Nevermind, will just test on train. XD

In [35]:
rf.score(train_data_vectors, train['sentiment'])

0.99284

In [36]:
# Not bad XD